In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install cartopy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 26.1 MB/s eta 0:00:00


In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

The **climatology** for **pre-monsoon (MAM)** and **monsoon (JJAS)** season was derived from ERA5 **mean surface net long-wave radiation flux and mean surface net short-wave radiation flux**  data (1981–2010)

In [ ]:
ds=xr.open_dataset('/content/drive/MyDrive/AP_HW/ERA5 Data/Mean_radiation/net_sw_lw-2024.nc')
ds

<xarray.Dataset> Size: 2GB
Dimensions:     (valid_time: 2928, latitude: 209, longitude: 381)
Coordinates:
    expver      (valid_time) object 23kB ...
  * valid_time  (valid_time) datetime64[ns] 23kB 2024-01-01 ... 2024-12-31T21...
  * longitude   (longitude) float64 3kB 25.0 25.25 25.5 ... 119.5 119.8 120.0
  * latitude    (latitude) float64 2kB 52.0 51.75 51.5 51.25 ... 0.5 0.25 0.0
    number      int32 4B ...
Data variables:
    msnlwrf     (valid_time, latitude, longitude) float32 933MB ...
    msnswrf     (valid_time, latitude, longitude) float32 933MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-03-26T09:59 GRIB to CDM+CF via cfgrib-0.9.1...

# MAM Climatology

In [ ]:
import os
from tqdm.auto import tqdm
import datetime

# Print process start time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))

# List to store yearly datasets
nr_list=[]

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/Mean_radiation'

# Loop over all .nc files in directory
for filename in tqdm(sorted(os.listdir(directory_path))):
    if filename.endswith('.nc'):
        # Open dataset
        file_path = os.path.join(directory_path, filename)
        year = filename[10:-3]
        ds = xr.open_dataset(file_path)

        # Define MAM period
        start_time = str(np.datetime64(f'{year}-03-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
        end_time = str(np.datetime64(f'{year}-05-31T17:00:00.000000000'))
        time_range = slice(start_time, end_time)
        ds = ds.sel(valid_time=time_range)

        # Adjust times to Bangladesh timezone
        ds['valid_time'] = ds['valid_time'] + np.timedelta64(6, 'h')

        # Yearly mean
        ds = ds.groupby('valid_time.year').mean(dim='valid_time', skipna=True)

        # Add to list
        nr_list.append(ds)

# Concatenate all years
nr_ds = xr.concat(nr_list, dim='year')

# Compute climatology
nr_clim_ds = nr_ds.sel(year=slice(1981, 2010))
nr_climatology = nr_clim_ds.mean(dim='year', skipna=True)
nr_climatology.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/radiation/net_sw_lw_climatology-mam.nc')  # Save to NetCDF

# Print process end time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))


2025-06-21 10:52:55.193604


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 11:06:33.154744


# JJAS Climatology

In [ ]:
import os
from tqdm.auto import tqdm
import datetime

# Print process start time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))

# List to store yearly datasets
nr_list = []

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/Mean_radiation'

# Loop over all .nc files in directory
for filename in tqdm(sorted(os.listdir(directory_path))):
    if filename.endswith('.nc'):
        # Open dataset
        file_path = os.path.join(directory_path, filename)
        year = filename[10:-3]
        ds = xr.open_dataset(file_path)

        # Define MAM period
        start_time = str(np.datetime64(f'{year}-06-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
        end_time = str(np.datetime64(f'{year}-09-30T17:00:00.000000000'))
        time_range = slice(start_time, end_time)
        ds = ds.sel(valid_time=time_range)  # Select time range

        # Adjust times to Bangladesh timezone
        ds['valid_time'] = ds['valid_time'] + np.timedelta64(6, 'h')

        # Yearly mean
        ds = ds.groupby('valid_time.year').mean(dim='valid_time', skipna=True)  # Yearly mean

        # Append to list
        nr_list.append(ds)

# Concatenate all years
nr_ds = xr.concat(nr_list, dim='year')

# Compute climatology
nr_clim_ds = nr_ds.sel(year=slice(1981, 2010))
nr_climatology = nr_clim_ds.mean(dim='year', skipna=True)
nr_climatology.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/radiation/net_sw_lw_climatology-jjas.nc')  # Save to NetCDF

# Print process end time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))


2025-06-21 11:06:33.167396


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 11:20:46.060528
